In [2]:
from dotenv import load_dotenv
load_dotenv()

True

Real-time playback

In [3]:
import os
import wave
from time import sleep

import pyaudio
import requests


def say(input):
    url = "https://api.openai.com/v1/audio/speech"
    headers = {
        "Authorization": f'Bearer {os.getenv("OPENAI_API_KEY")}',
    }

    data = {
        "model": "tts-1",
        "input": input,
        "voice": "shimmer",
        "response_format": "wav",
    }

    response = requests.post(url, headers=headers, json=data, stream=True)

    if response.ok:
        with wave.open(response.raw, "rb") as wf:

            def callback(in_data, frame_count, time_info, status):
                data = wf.readframes(frame_count)
                # If len(data) is less than requested frame_count, PyAudio automatically
                # assumes the stream is finished, and the stream stops.
                return (data, pyaudio.paContinue)

            p = pyaudio.PyAudio()
            stream = p.open(
                format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True,
                stream_callback=callback,
            )

            # Wait for stream to finish
            while stream.is_active():
                sleep(1)
            stream.close()
            p.terminate()
    else:
        response.raise_for_status()

In [14]:
say("This is a test of the speech synthesis API")